In [1]:
import pandas as pd
import numpy as np
import random
random.seed(1)

In [2]:
### global variable (need to train)
mu = 2/3

In [3]:
# Defining some sample data for artists, albums, tracks, and genres
artists = ["Artist A", "Artist B", "Artist C", "Artist D", "Artist E"]
albums = ["Album 1", "Album 2", "Album 3", "Album 4", "Album 5"]
tracks = ["Track 1", "Track 2", "Track 3", "Track 4", "Track 5"]
genres = ["Pop", "Rock", "Jazz", "Kpop", "Classical"]
random.seed(1)
# Function to create a unique song
def create_song(id):
    return {
        "Song ID": id,
        "Artist": random.choice(artists),
        "Album": random.choice(albums),
        "Track": random.choice(tracks),
        "Genre": random.choice(genres)
    }

# Creating 100 unique songs
songs_data = [create_song(i) for i in range(1, 101)]
songs_df = pd.DataFrame(songs_data)
songs_df

,Song ID,Artist,Album,Track,Genre
0,1,Artist B,Album 5,Track 1,Jazz
1,2,Artist A,Album 4,Track 4,Kpop
2,3,Artist D,Album 2,Track 1,Kpop
3,4,Artist A,Album 4,Track 4,Classical
4,5,Artist A,Album 4,Track 3,Rock
...,...,...,...,...,...
95,96,Artist C,Album 1,Track 3,Pop
96,97,Artist D,Album 1,Track 5,Jazz
97,98,Artist B,Album 4,Track 3,Pop
98,99,Artist C,Album 2,Track 3,Classical


In [4]:
songs_data

[{'Song ID': 1,
  'Artist': 'Artist B',
  'Album': 'Album 5',
  'Track': 'Track 1',
  'Genre': 'Jazz'},
 {'Song ID': 2,
  'Artist': 'Artist A',
  'Album': 'Album 4',
  'Track': 'Track 4',
  'Genre': 'Kpop'},
 {'Song ID': 3,
  'Artist': 'Artist D',
  'Album': 'Album 2',
  'Track': 'Track 1',
  'Genre': 'Kpop'},
 {'Song ID': 4,
  'Artist': 'Artist A',
  'Album': 'Album 4',
  'Track': 'Track 4',
  'Genre': 'Classical'},
 {'Song ID': 5,
  'Artist': 'Artist A',
  'Album': 'Album 4',
  'Track': 'Track 3',
  'Genre': 'Rock'},
 {'Song ID': 6,
  'Artist': 'Artist E',
  'Album': 'Album 1',
  'Track': 'Track 3',
  'Genre': 'Pop'},
 {'Song ID': 7,
  'Artist': 'Artist A',
  'Album': 'Album 1',
  'Track': 'Track 5',
  'Genre': 'Pop'},
 {'Song ID': 8,
  'Artist': 'Artist D',
  'Album': 'Album 2',
  'Track': 'Track 4',
  'Genre': 'Pop'},
 {'Song ID': 9,
  'Artist': 'Artist E',
  'Album': 'Album 2',
  'Track': 'Track 4',
  'Genre': 'Kpop'},
 {'Song ID': 10,
  'Artist': 'Artist E',
  'Album': 'Album 2',

In [5]:
# 總共有這麼多首不一樣的歌
# Grouping the song data by 'Artist', 'Album', and 'Track', and listing the genres for each group
grouped_songs = songs_df.groupby(['Artist', 'Album', 'Track'])['Genre'].apply(list).reset_index()
grouped_songs

,Artist,Album,Track,Genre
0,Artist A,Album 1,Track 1,"[Kpop, Rock]"
1,Artist A,Album 1,Track 3,[Classical]
2,Artist A,Album 1,Track 4,[Pop]
3,Artist A,Album 1,Track 5,[Pop]
4,Artist A,Album 2,Track 2,[Classical]
...,...,...,...,...
65,Artist E,Album 4,Track 3,[Kpop]
66,Artist E,Album 4,Track 4,[Jazz]
67,Artist E,Album 4,Track 5,[Rock]
68,Artist E,Album 5,Track 1,[Kpop]


### artist, album, track, genre 的 sim 計算方式

artist標記規則:
1. item i 和 j 假如是相同的artist 相似度就是 1
2. Besides, we sort i’s neighbors in descending order by their popularity.

album標記規則
1. 假如 歌手 專輯 就是 1 
2. 假如 歌手 一樣 就是 2
3. 都不一樣 就是 0

track標記規則:
1. 假如 歌手 專輯 歌曲都一樣 就是 1 或者是 歌手 專輯 一樣 歌名不一樣但音樂種類都一樣 
2. 假如 歌手 專輯 都一樣 就是 2
3. 假如 歌手 一樣 就是 3
4. 都不一樣 就是 0

genre標記規則
1. 與artist類似
2. we also consider two genres are similar if they belong to a track’s (album) genre sets together.

In [6]:
# Updated function to include comparison within the same group and to check similarity in Artist, Album, and Track
# track標記規則
# 假如 歌手 專輯 歌曲都一樣 就是 1 或者是 歌手 專輯 一樣 歌名不一樣但音樂種類都一樣 
# 假如 歌手 專輯 都一樣 就是 2
# 假如 歌手 一樣 就是 3
# 都不一樣 就是 0
def track_compare_songs_and_label(song1, song2):
    # Comparing Artist, Album, and Track
    if song1['Artist'] == song2['Artist'] and song1['Album'] == song2['Album'] and song1['Track'] == song2['Track']:
        similarity_label = 1
    elif song1['Artist'] == song2['Artist'] and song1['Album'] == song2['Album']:
        similarity_label = 2
    elif song1['Artist'] == song2['Artist']: 
        similarity_label = 3
    else:
        similarity_label = 0

    # Comparing genres
    genres1 = set(song1['Genre'])
    genres2 = set(song2['Genre'])
    intersection = genres1.intersection(genres2)
    union = genres1.union(genres2)

    return len(intersection), len(union), similarity_label


# album標記規則
# 假如 歌手 專輯 就是 1 
# 假如 歌手 一樣 就是 2
# 都不一樣 就是 0
def album_compare_songs_and_label(song1, song2):
    if song1['Artist'] == song2['Artist'] and song1['Album'] == song2['Album']:
        similarity_label = 1
    elif song1['Artist'] == song2['Artist']: 
        similarity_label = 2
    else:
        similarity_label = 0
        
    return similarity_label

# Preparing a list to store the updated comparison results
comparison_results = []

# Comparing each pair of songs including self comparison
for i in range(len(grouped_songs)):
    for j in range(i, len(grouped_songs)):  # Changed to start from i for self comparison
        song1 = grouped_songs.iloc[i]
        song2 = grouped_songs.iloc[j]
        inter, uni, track_label = track_compare_songs_and_label(song1, song2)
        album_label = album_compare_songs_and_label(song1, song2)
        comparison_results.append({
            'Song 1': (song1['Artist'], song1['Album'], song1['Track']),
            'Song 2': (song2['Artist'], song2['Album'], song2['Track']),
            'Intersection': inter,
            'Union': uni,
            'track Similarity Label': track_label,
            'album Similarity Label': album_label
        })

# Converting the updated comparison results into a DataFrame
comparison_df = pd.DataFrame(comparison_results)

comparison_df['track_sim'] = comparison_df.apply(
    lambda row: 1 if row['track Similarity Label'] == 1 
               else (2 + (row['Intersection'] / row['Union'])) / 3  if row['track Similarity Label'] == 2
               else (1 + (row['Intersection'] / row['Union'])) / 2  if row['track Similarity Label'] == 3
               else mu * (row['Intersection'] / row['Union'])  if row['track Similarity Label'] == 0
               else row['track Similarity Label'],
    axis=1
)

comparison_df['album_sim'] = comparison_df.apply(
    lambda row: 1 if row['album Similarity Label'] == 1 
               else (1 + (row['Intersection'] / row['Union'])) / 2  if row['album Similarity Label'] == 2
               else mu * (row['Intersection'] / row['Union'])  if row['album Similarity Label'] == 0
               else row['Similarity Label'],
    axis=1
)

comparison_df

,Song 1,Song 2,Intersection,Union,track Similarity Label,album Similarity Label,track_sim,album_sim
0,"(Artist A, Album 1, Track 1)","(Artist A, Album 1, Track 1)",2,2,1,1,1.000000,1.0
1,"(Artist A, Album 1, Track 1)","(Artist A, Album 1, Track 3)",0,3,2,1,0.666667,1.0
2,"(Artist A, Album 1, Track 1)","(Artist A, Album 1, Track 4)",0,3,2,1,0.666667,1.0
3,"(Artist A, Album 1, Track 1)","(Artist A, Album 1, Track 5)",0,3,2,1,0.666667,1.0
4,"(Artist A, Album 1, Track 1)","(Artist A, Album 2, Track 2)",0,3,3,2,0.500000,0.5
...,...,...,...,...,...,...,...,...
2480,"(Artist E, Album 4, Track 5)","(Artist E, Album 5, Track 1)",0,2,3,2,0.500000,0.5
2481,"(Artist E, Album 4, Track 5)","(Artist E, Album 5, Track 5)",0,3,3,2,0.500000,0.5
2482,"(Artist E, Album 5, Track 1)","(Artist E, Album 5, Track 1)",1,1,1,1,1.000000,1.0
2483,"(Artist E, Album 5, Track 1)","(Artist E, Album 5, Track 5)",1,2,2,1,0.833333,1.0


In [7]:
random.seed(1)
# Function to create a random list of songs and their ratings for a person
def create_person_songs_and_ratings(person_id, song_ids):
    # Each person has listened to a random number of songs
    num_songs = random.randint(5, 15)  # Each person listens to 5 to 15 songs
    listened_songs = random.sample(song_ids, num_songs)

    # Generating ratings for each song
    ratings = [random.randint(1, 10) for _ in range(num_songs)]

    # Creating a list of dictionaries for each song and its rating
    return [{"Person ID": person_id, "Song ID": song_id, "Rating": rating} for song_id, rating in zip(listened_songs, ratings)]

# Generating data for 100 people
people_songs_ratings = []
song_ids = list(range(1, 101))  # IDs of the 100 songs

for person_id in range(1, 101):
    people_songs_ratings.extend(create_person_songs_and_ratings(person_id, song_ids))

# Converting the data into a Pandas DataFrame
ratings_df = pd.DataFrame(people_songs_ratings)
ratings_df

,Person ID,Song ID,Rating
0,1,73,8
1,1,98,7
2,1,9,4
3,1,33,2
4,1,16,8
...,...,...,...
983,100,43,1
984,100,79,3
985,100,93,5
986,100,46,9
